### Part 1: data acquisition by scraping lyrics from lyrics.com, parsing and cleaning them, and exporting all into a .csv

In [ ]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import os

headers = {"User-agent": "Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko 20100101 Firefox/4.0b10"}

In [ ]:
#David Bowie
URL = "https://www.lyrics.com/artist/David-Bowie/3753"

headers = {"User-agent": "Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko 20100101 Firefox/4.0b10"}

response = requests.get(url = URL, headers = headers)

bowie_htm = response.text

f = open("bowie_htm.txt", "w")
f.write(response.text)
f.close()

with open ("bowie_htm.txt", "r") as bowie:
    bowie = bowie.read()

link = r'<a href=\"(/lyric/\d+/David\+Bowie/[\d\w\+]+)\">'
bowie_links = re.findall(link, bowie)
bowie_links = pd.DataFrame({"links":bowie_links})
bowie_links["links"] = "https://www.lyrics.com" + bowie_links["links"]
bowie_links["songs"] = bowie_links["links"].str[50:]
bowie_links["songs"] = bowie_links["songs"].str.replace("+", "_").str.lower()
bowie_links.drop_duplicates(subset = "songs", inplace = True)
bowie_links = bowie_links.iloc[1:151]

for _, row in bowie_links.iterrows():
    response = requests.get(url = row["links"], headers = headers)
    f = open("/data/bowie/"+row["songs"]+".txt", "w")
    f.write(response.text)
    f.close()

bowie_links.to_csv("/data/bowie_links.csv")

bowie_links["lyrics"] = np.nan
bowie_links["artist"] = "David Bowie"

for fn in os.listdir("/data/bowie/"):
    with open("/data/bowie/" + fn) as song_lyrics_file:
        song_lyrics = song_lyrics_file.read()
    song_name = fn[:-4]
    
    song_lyrics = BeautifulSoup(song_lyrics, "html.parser")
    song_lyrics = song_lyrics.find(class_ = "lyric-body")
    
    if song_lyrics is not None:
        clean_song_lyrics = song_lyrics.get_text(" ", strip = True)

    bowie_links.loc[bowie_links.songs == song_name, "lyrics"] = clean_song_lyrics

bowie_links = bowie_links.iloc[1:101]

In [ ]:
#Tina Turner
URL = "https://www.lyrics.com/artist/Tina-Turner/5714"

headers = {"User-agent": "Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko 20100101 Firefox/4.0b10"}

response = requests.get(url = URL, headers = headers)

turner_htm = response.text

f = open("turner_htm.txt", "w")
f.write(response.text)
f.close()

with open ("turner_htm.txt", "r") as turner:
    turner = turner.read()

link = r'<a href=\"(/lyric/\d+/Tina\+Turner/[\d\w\+]+)\">'
turner_links = re.findall(link, turner)
turner_links = pd.DataFrame({"links":turner_links})
turner_links["links"] = "https://www.lyrics.com" + turner_links["links"]
turner_links["songs"] = turner_links["links"].str[50:]
turner_links["songs"] = turner_links["songs"].str.replace("+", "_").str.lower()
turner_links.drop_duplicates(subset = "songs", inplace = True)
turner_links = turner_links.iloc[1:151]

for _, row in turner_links.iterrows():
    response = requests.get(url = row["links"], headers = headers)
    f = open("/data/turner/"+row["songs"]+".txt", "w")
    f.write(response.text)
    f.close()

turner_links.to_csv("/data/turner_links.csv")

turner_links["lyrics"] = np.nan
turner_links["artist"] = "Tina Turner"

for fn in os.listdir("/data/turner/"):
    with open("/data/turner/" + fn) as song_lyrics_file:
        song_lyrics = song_lyrics_file.read()
    song_name = fn[:-4]

    song_lyrics = BeautifulSoup(song_lyrics, "html.parser")
    song_lyrics = song_lyrics.find(class_ = "lyric-body")

    if song_lyrics is not None:
        clean_song_lyrics = song_lyrics.get_text(" ", strip = True)

    turner_links.loc[turner_links.songs == song_name, "lyrics"] = clean_song_lyrics

turner_links = turner_links.iloc[1:101]

In [ ]:
lyrics = pd.concat([bowie_links, turner_links], axis = 0)
lyrics.dropna(inplace = True)
lyrics.to_csv("/data/lyrics.csv")